In [1]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import metrics as sk
from sklearn.metrics import confusion_matrix
import networkx as nx
from torch_geometric.typing import SparseTensor
from GraphRicciCurvature.FormanRicci import FormanRicci
from torch_geometric.nn.conv.gcn_conv import gcn_norm
import torch.nn.functional as F
import torch
from torchviz import make_dot
import seaborn as sns
import os
import time

In [2]:
TRAIN_IMAGES_PATH = './data/images/train'
TEST_IMAGES_PATH = './data/images/test'
VAL_IMAGES_PATH = './data/images/val'

TRAIN_NPZ_FILE = './data/images/npz/train_images.npz'
TEST_NPZ_FILE = './data/images/npz/test_images.npz'
VAL_NPZ_FILE = './data/images/npz/val_images.npz'

NUM_FEATURES = 224 * 224 * 3

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_type = "GIN"
max_hop = 1
layers = 3
hidden_channels = 2048
dropout = 0.5
batch_norm = True
lr = 0.0005
num_batch = 3
num_epoch = 400
multilabel = False
do_evaluation = True
residual = True
print_result = True
aggregations_flow = 100
max_communities = 1000
remove_edges = 0
make_unbalanced = True
dense = True
topological_measure = "none"

## Load Data

In [3]:
def load_npz_as_tensors(file_path):
    """
    Load the .npz files as tensors

    Args:
        file_path (string): To get the .npz file and load it as a tensor
    """
    data = np.load(file_path, allow_pickle=True)

    images = data['images']
    labels = data['labels']

    images_tensor = torch.tensor(images, dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.long)

    return images_tensor, labels_tensor

## Process Data and Save Masked Layers

In [4]:
def process_data(sparsity):
    """
    Process and save the data in both sparse and dense formats.

    Args:
        sparsity (bool): Whether to process the dataset as sparse or dense
    """
    
    # Load training data
    train_images, train_labels = load_npz_as_tensors(TRAIN_NPZ_FILE)
    train_data = train_images.reshape(train_images.shape[0], NUM_FEATURES)
    num_train = train_images.shape[0]

    # Load validation data
    val_images, val_labels = load_npz_as_tensors(VAL_NPZ_FILE)
    val_data = val_images.reshape(val_images.shape[0], NUM_FEATURES)
    num_val = val_images.shape[0]

    # Load test data
    test_images, test_labels = load_npz_as_tensors(TEST_NPZ_FILE)
    test_data = test_images.reshape(test_images.shape[0], NUM_FEATURES)
    num_test = test_images.shape[0]

    # Concatenate train, validation, and test data
    num_data = num_train + num_val + num_test
    data_feat = np.concatenate((train_data, val_data, test_data), axis=0)
    data_label = np.concatenate((train_labels, val_labels, test_labels), axis=0).reshape(-1)

    # Construct and scale adjacency matrix
    adj_matrix = sk.pairwise.cosine_similarity(data_feat, data_feat)
    adj_matrix = (adj_matrix - adj_matrix.min())/(adj_matrix.max()-adj_matrix.min())

    # Apply sparsity thresholds
    threshold = 0.977 if sparsity else 0.970

    adj_matrix = adj_matrix > threshold

    # Generate masks
    train_mask = np.zeros(num_data, dtype=bool)
    train_mask[:num_train] = True
    val_mask = np.zeros(num_data, dtype=bool)
    val_mask[num_train:num_train + num_val] = True
    test_mask = np.zeros(num_data, dtype=bool)
    test_mask[num_train + num_val:] = True

    # Save masks, features, labels, and edge index
    suffix = 'sparse' if sparsity else 'dense'
    base_path = f"./data/npy/{suffix}"

    np.save(f"{base_path}/train_mask.npy", train_mask)
    np.save(f"{base_path}/val_mask.npy", val_mask)
    np.save(f"{base_path}/test_mask.npy", test_mask)
    np.save(f"{base_path}/data_feat.npy", data_feat)
    np.save(f"{base_path}/data_label.npy", data_label)

    # Generate and save edge index
    edge_index = np.array([[i, j] for i in range(num_data) for j in range(num_data) if i != j and adj_matrix[i, j]])
    np.save(f"{base_path}/edge_index.npy", edge_index)

    print(f"View-({'Sparse' if sparsity else 'Dense'}) generated!")

In [5]:
# Call the function with sparsity parameter
#process_data(sparsity=True)
#process_data(sparsity=False)

## Load Data, Masks, and Print Statistics

In [6]:
def print_statistics(features, labels, edge_index, train_mask, val_mask, test_mask):
    """
    Print statistics of the dataset

    Args:
        features (np.ndarray): Node features
        labels (np.ndarray): Node labels
        edge_index (np.ndarray): Edge indices
        train_mask (torch.Tensor): Mask for training nodes
        val_mask (torch.Tensor): Mask for validation nodes
        test_mask (torch.Tensor): Mask for test nodes
    """
    print("=============== Dataset Types ==============")
    print(f"Type of features: {type(features)}")
    print(f"Type of labels: {type(labels)}")
    print(f"Type of edge_index: {type(edge_index)}")
    print(f"Type of train_mask: {type(train_mask)}")
    print(f"Type of val_mask: {type(val_mask)}")
    print(f"Type of test_mask: {type(test_mask)}")
    print("=============== Dataset Properties ==================")
    print(f"Total Nodes: {features.shape[0]}")
    print(f"Total Edges: {edge_index.shape[0]}")
    print(f"Number of Features: {features.shape[1]}")
    if labels.ndim == 1:
        print(f"Number of Labels: {labels.max() + 1}")
        print("Task Type: Multi-class Classification")
    else:
        print(f"Number of Labels: {labels.shape[1]}")
        print("Task Type: Multi-label Classification")
    print(f"Training Nodes: {train_mask.sum().item()}")
    print(f"Validation Nodes: {val_mask.sum().item()}")
    print(f"Testing Nodes: {test_mask.sum().item()}")
    print()

In [7]:
def get_dataset(sparse=True, balanced=True):
    """
    Load the dataset in either sparse or dense format

    Args:
        sparse (bool): Whether to load the sparse or dense version of the dataset
    """
    print(f"Loading Dataset")
    
    # Load masks
    suffix = 'sparse' if sparse else 'dense'
    
    train_mask = torch.tensor(np.load(f"./data/npy/{suffix}/train_mask.npy"))
    val_mask = torch.tensor(np.load(f"./data/npy/{suffix}/val_mask.npy"))
    test_mask = torch.tensor(np.load(f"./data/npy/{suffix}/test_mask.npy"))

    # Load labels
    labels = np.load(f"./data/npy/{suffix}/data_label.npy")

    # Load and normalize features
    features = np.load(f"./data/npy/{suffix}/data_feat.npy")
    features = sklearn.preprocessing.StandardScaler().fit_transform(features)

    # Load edge indices
    edge_index = np.load(f"./data/npy/{suffix}/edge_index.npy")

    # Print dataset statistics
    print_statistics(features, labels, edge_index, train_mask, val_mask, test_mask)

    if not balanced:
        all_labels = [0, 1, 2, 3, 4, 5]
        chosen_labels = [0, 1, 2, 3, 4, 5]

        print("[Before unbalancing] Class distribution in the training set:")
        for label in all_labels:
            count = np.sum(labels[train_mask] == label)
            print(f"Label {label}: {count} samples")
        
        print("[Before unbalancing] Class distribution in the validation set:")
        for label in all_labels:
            count = np.sum(labels[val_mask] == label)
            print(f"Label {label}: {count} samples")

        print("[Before unbalancing] Class distribution in the test set:")
        for label in all_labels:
            count = np.sum(labels[test_mask] == label)
            print(f"Label {label}: {count} samples")

        chosen_indices = np.where(np.isin(labels[train_mask], chosen_labels))[0]
        train_indices, test_indices = train_test_split(chosen_indices, test_size=0.8, stratify=labels[train_mask][chosen_indices])
                
        new_train_mask = torch.full_like(train_mask, False)
        new_train_mask[train_indices] = True

        for i, label in enumerate(labels):
            if label in chosen_labels and new_train_mask[i] == False:
                    train_mask[i] = False

        train_mask[train_indices] = True
        test_mask[test_indices] = True

        print("Class distribution in the training set:")
        for label in all_labels:
            count = np.sum(labels[train_mask] == label)
            print(f"Label {label}: {count} samples")

        print("Class distribution in the validation set:")
        for label in all_labels:
            count = np.sum(labels[val_mask] == label)
            print(f"Label {label}: {count} samples")

        print("Class distribution in the test set:")
        for label in all_labels:
            count = np.sum(labels[test_mask] == label)
            print(f"Label {label}: {count} samples")
            
    return features, labels, edge_index, train_mask, val_mask, test_mask

## Graphs and Graph Constructions

In [8]:
def construct_graph(x, y, edge_index, train_mask, val_mask, test_mask):
    """
    Construct a NetworkX graph from node features, labels, and edge information.

    Args:
        x (np.ndarray or torch.Tensor): Node features with shape (num_nodes, feature_dim).
        y (np.ndarray or torch.Tensor): Node labels with shape (num_nodes,).
        edge_index (torch.Tensor or list of tuples): Edge indices either in PyTorch Geometric format (2xN tensor) or standard edge list format.
        train_mask (np.ndarray or list): Boolean mask indicating training nodes.
        val_mask (np.ndarray or list): Boolean mask indicating validation nodes.
        test_mask (np.ndarray or list): Boolean mask indicating test nodes.

    Returns:
        nx.Graph: A NetworkX graph with nodes having attributes for features, labels, and masks, and edges with default weights.
    """
    # Construct NetworkX Graph
    nodes = [i for i in range(x.shape[0])]

    G = nx.Graph()

    # Add nodes with attributes
    for i in nodes:
        G.add_node(i, x=x[i], y=y[i], train=train_mask[i], val=val_mask[i], test=test_mask[i])
    
    # Handle edge_index input (PyTorch Geometric format)
    if isinstance(edge_index, torch.Tensor) and edge_index.dim() == 2 and edge_index.shape[0] == 2:
        edge_list = edge_index.t().tolist()
    else:
        # Assuming edge_index is in standard edge list format
        edge_list = edge_index

    # Add edges with a default weight of 1
    weighted_edges = [(edge[0], edge[1], 1) for edge in edge_list]
    G.add_weighted_edges_from(weighted_edges)

    return G

In [9]:
def split_graph(G, multilabel = True):
    """
    Split the graph into training, validation, and test sets.

    Args:
        G (nx.Graph): The input NetworkX graph with node attributes specifying train, val, and test masks.
        multilabel (bool): Flag indicating if the graph is multilabel. Defaults to True.

    Returns:
        tuple: A tuple containing:
            - x_train (np.ndarray or torch.Tensor): Node features for training nodes.
            - y_train (np.ndarray or torch.Tensor): Node labels for training nodes.
            - edge_train (np.ndarray or torch.Tensor): Edge indices for training nodes.
            - train_mask (np.ndarray or torch.Tensor): Boolean mask for training nodes.
            - x_val (np.ndarray or torch.Tensor): Node features for validation nodes.
            - y_val (np.ndarray or torch.Tensor): Node labels for validation nodes.
            - edge_val (np.ndarray or torch.Tensor): Edge indices for validation nodes.
            - val_mask (np.ndarray or torch.Tensor): Boolean mask for validation nodes.
            - x_test (np.ndarray or torch.Tensor): Node features for test nodes.
            - y_test (np.ndarray or torch.Tensor): Node labels for test nodes.
            - edge_test (np.ndarray or torch.Tensor): Edge indices for test nodes.
            - test_mask (np.ndarray or torch.Tensor): Boolean mask for test nodes.
    """
    print("Splitting Graph...")
    print("=============== Graph Splitting ===============")
    
    # Get complete test graph
    x_test, y_test, edge_test, _, _, test_mask = convert_graph_to_tensor(G, multilabel=multilabel)
    
    print(f"Unlabeled + Test + Validation + Training graph nodes: {x_test.shape[0]}")
    print(f"Unlabeled + Test + Validation + Training graph edges: {edge_test.shape[0]}")
    print(f"Total test nodes: {test_mask.sum()}")
    
    # Get training + val graph
    # remove all test nodes
    test_nodes = []
    for node in G.nodes(data=True):
        if node[1]['test']:
            test_nodes.append(node[0])
    G.remove_nodes_from(test_nodes)
    G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default')
    x_val, y_val, edge_val, _, val_mask, _ = convert_graph_to_tensor(G, multilabel=multilabel)
    
    print(f"Unlabeled + Validation + Training graph nodes: {x_val.shape[0]}")
    print(f"Unlabeled + Validation + Training graph edges: {edge_val.shape[0]}")
    print(f"Total val nodes: {val_mask.sum()}")
    # Get training graph
    # remove all val nodes
    val_nodes = []
    for node in G.nodes(data=True):
        if node[1]['val']:
            val_nodes.append(node[0])
    G.remove_nodes_from(val_nodes)
    G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default')
    
    x_train, y_train, edge_train, train_mask, _, _ = convert_graph_to_tensor(G, multilabel = multilabel)
    
    print(f"Unlabeled + Training graph nodes: {x_train.shape[0]}")
    print(f"Unlabeled + Training graph edges: {edge_train.shape[0]}")
    print(f"Total train nodes: {train_mask.sum()}")
    print()
    
    return (x_train, y_train, edge_train, train_mask, x_val, y_val, edge_val, 
            val_mask, x_test, y_test, edge_test, test_mask)

In [10]:
def convert_graph_to_tensor(G, multilabel = True):
    """
    Convert a NetworkX graph into tensors or numpy arrays for use in machine learning models.

    Args:
        G (nx.Graph): The input NetworkX graph with node attributes for features, labels, and masks.
        multilabel (bool): Flag indicating if the graph is multilabel. Defaults to True.

    Returns:
        tuple: A tuple containing:
            - x (np.ndarray or torch.Tensor): Node features.
            - y (np.ndarray or torch.Tensor): Node labels.
            - edge_index (np.ndarray or torch.Tensor): Edge indices.
            - train_mask (np.ndarray or torch.Tensor): Boolean mask for training nodes.
            - val_mask (np.ndarray or torch.Tensor): Boolean mask for validation nodes.
            - test_mask (np.ndarray or torch.Tensor): Boolean mask for test nodes.
    """
    x = np.empty((G.number_of_nodes(),G.nodes[0]['x'].shape[0]))
    
    if multilabel:
        y = np.empty((G.number_of_nodes(),G.nodes[0]['y'].shape[0]),dtype = 'int')
    else:
        y = np.empty((G.number_of_nodes(),),dtype = 'int')
        
    edge_index = np.array([edge for edge in G.edges()])
    train_mask = np.empty((G.number_of_nodes(),),dtype = 'bool')
    val_mask = np.empty((G.number_of_nodes(),),dtype = 'bool')
    test_mask = np.empty((G.number_of_nodes(),),dtype = 'bool')
    
    for node in G.nodes(data=True):
        x[node[0],:] = node[1]['x']
        if multilabel:
            y[node[0],:] = node[1]['y']
        else:
            y[node[0]] = node[1]['y']
        
        train_mask[node[0]] = node[1]['train']
        val_mask[node[0]] = node[1]['val']
        test_mask[node[0]] = node[1]['test']
    
    return x, y, edge_index, train_mask, val_mask, test_mask

In [11]:
def construct_normalized_adj(edge_index, num_nodes):
    """
    Construct a normalized adjacency matrix from edge indices.

    Args:
        edge_index (np.ndarray or torch.Tensor): Edge indices in the format [2, num_edges].
        num_nodes (int): Number of nodes in the graph.

    Returns:
        SparseTensor: Normalized adjacency matrix with self-loops added and GCN normalization applied.
    """
    edge_index = torch.tensor(edge_index)
    edge_index = torch.transpose(edge_index,0,1)
    edge_index_flip = torch.flip(edge_index,[0]) # re-adds flipped edges that were removed by networkx
    edge_index = torch.cat((edge_index, edge_index_flip), 1)
    adj = SparseTensor(row=edge_index[0], col=edge_index[1], sparse_sizes=(num_nodes,num_nodes))
    adj = adj.set_diag() # adding self loops
    adj = gcn_norm(adj, add_self_loops=False) # normalization

    return adj

## Get Metrics of the Training

In [12]:
def logit_to_label(out):
    """
    Convert logits to predicted labels using argmax

    Args:
        out (torch.Tensor): Logits tensor

    Returns:
        torch.Tensor: Predicated labels
    """
    return out.argmax(dim=1)

In [13]:
def metrics(logits, y):
    """
    Calculate classification metrics.

    Args:
        logits (torch.Tensor): Model output logits.
        y (torch.Tensor): True labels.

    Returns:
        tuple: (accuracy, micro F1 score, sensitivity, specificity)
    """
    if y.dim() == 1: # Multi-class
        y_pred = logit_to_label(logits)
        cm = confusion_matrix(y.cpu(),y_pred.cpu())
        FP = cm.sum(axis=0) - np.diag(cm)  
        FN = cm.sum(axis=1) - np.diag(cm)
        TP = np.diag(cm)
        TN = cm.sum() - (FP + FN + TP)
    
        acc = np.diag(cm).sum() / cm.sum()
        micro_f1 = acc # micro f1 = accuracy for multi-class
        sens = TP.sum() / (TP.sum() + FN.sum())
        spec = TN.sum() / (TN.sum() + FP.sum())
    
    else: # Multi-label
        y_pred = logits >= 0
        y_true = y >= 0.5
        
        tp = int((y_true & y_pred).sum())
        tn = int((~y_true & ~y_pred).sum())
        fp = int((~y_true & y_pred).sum())
        fn = int((y_true & ~y_pred).sum())
        
        acc = (tp + tn)/(tp + fp + tn + fn)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        micro_f1 = 2 * (precision * recall) / (precision + recall)
        sens = (tp)/(tp + fn)
        spec = (tn)/(tn + fp)
        
    return acc, micro_f1, sens, spec


In [14]:
def edge_metric_compute(metric, graph):
    """
    Compute a specified edge metric by averaging the provided node metrics for each edge.

    Args:
        metric (dict): A dictionary where keys are nodes and values are the metric values for the nodes.
        graph (networkx.Graph): A NetworkX graph object.

    Returns:
        dict: A dictionary where keys are edges and values are the average metric values of the nodes incident to the edge.
    """
    edges = {}
    
    # Iterate over edges
    for edge in graph.edges():
        u, v = edge
        
        metric_u = metric[u]
        metric_v = metric[v]
        
        avg_centrality = (metric_u + metric_v) / 2
        
        edges[edge] = avg_centrality
    
    return edges

In [15]:
def edge_degree_centrality(graph):
    """
    Compute the degree centrality for each edge in the graph.

    Args:
        graph (networkx.Graph): A NetworkX graph object.

    Returns:
        dict: A dictionary where keys are edges and values are the average degree of the nodes incident to the edge.
    """
    edge_degree = {}
    
    # Iterate over edges
    for edge in graph.edges():
        u, v = edge
        
        # Compute degrees of the nodes incident to the edge
        degree_u = graph.degree(u)
        degree_v = graph.degree(v)
        
        # Calculate average degree
        avg_degree = (degree_u + degree_v) / 2
        
        # Assign average degree as edge degree centrality
        edge_degree[edge] = avg_degree
    
    return edge_degree


In [16]:
def calculate_homophily_ratios(adj, x, y):
    """
    Calculate homophily ratios based on feature and label similarities.

    Args:
        adj (scipy.sparse.coo_matrix): Adjacency matrix in COO format.
        x (numpy.ndarray or torch.Tensor): Node features.
        y (numpy.ndarray or torch.Tensor): Node labels.

    Returns:
        list: List of homophily ratios for each edge in the adjacency matrix.
    """
    homophily_ratios = []
    
    # Extract COO format components
    row, col, _ = adj.coo()
    
    for r, c in zip(row.tolist(), col.tolist()):
        # Extract features and labels using row and col indices
        x1, y1 = x[r], y[r]
        x2, y2 = x[c], y[c]
        
        # Calculate similarity in features (assuming features are numpy arrays)
        feature_similarity = np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))
        
        # Calculate similarity in labels
        label_similarity = 1 if y1 == y2 else 0
        
        # Homophily ratio can be a combination of both similarities
        homophily_ratio = 0.5 * feature_similarity + 0.5 * label_similarity
        
        homophily_ratios.append(homophily_ratio)
    
    return homophily_ratios

## Visualization

In [17]:
def plot_deltas(deltas, num_nodes_to_plot=5):
    """
    Plot the feature variation (delta) over aggregation steps for a specified number of nodes.

    Parameters:
    - deltas (list of np.array): List where each element is an array of deltas for a specific node.
    - num_nodes_to_plot (int): Number of nodes to plot. Defaults to 5.

    This function creates a line plot where each line represents the feature variation over aggregation steps for each node.
    """
    plt.figure(figsize=(10, 6))
    for i in range(num_nodes_to_plot):
        plt.plot(deltas[i], label=f'Node {i}')
    plt.xlabel('Aggregation Step')
    plt.ylabel('Feature Variation (Delta)')
    plt.title('Feature Variation Over Aggregation Steps')
    plt.legend()
    plt.show()

In [18]:
def plot_mse_results(mse_results, filename='gin_mse_results.png'):
    """
    Plot the Mean Squared Error (MSE) results for each node and save the plot as an image.

    Parameters:
    - mse_results (list of float): List of MSE values for each node.
    - filename (str): Name of the file where the plot will be saved.

    This function creates a bar plot where each bar represents the MSE for a specific node and saves the plot to a file.
    """
    num_nodes = len(mse_results)
    nodes = np.arange(num_nodes)
    filename = "./results/gnn/images/" + filename

    plt.figure(figsize=(10, 6))
    plt.bar(nodes, mse_results, color='skyblue')
    plt.xlabel('Nodes')
    plt.ylabel('Mean Squared Error (MSE)')
    plt.title('Mean Squared Error for Each Node')
    plt.xticks(nodes)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.savefig(filename)
    plt.show()

In [19]:
def calculate_homophily_ratios_for_graph(graph):
    """
    Calculate the homophily ratios for all edges in the graph.

    Parameters:
    - graph (networkx.Graph): The input graph where each node has 'x' (features) and 'y' (label) attributes.

    Returns:
    - homophily_ratios (list of float): List of homophily ratios for each edge in the graph.

    Homophily ratio is calculated as a combination of feature similarity and label similarity between connected nodes.
    """
    homophily_ratios = []

    for edge in graph.edges():
        node1, node2 = edge
        # Extract features and labels for both nodes
        x1, y1 = graph.nodes[node1]['x'], graph.nodes[node1]['y']
        x2, y2 = graph.nodes[node2]['x'], graph.nodes[node2]['y']
        
        # Calculate similarity in features (using cosine similarity)
        feature_similarity = np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))
        
        # Calculate similarity in labels
        label_similarity = 1 if y1 == y2 else 0
        
        # Compute homophily ratio as a weighted combination of feature and label similarity
        homophily_ratio = 0.5 * feature_similarity + 0.5 * label_similarity
        
        homophily_ratios.append(homophily_ratio)
    
    return homophily_ratios

In [20]:
def plot_homophily_distributions(graphs, stages, dataset):
    """
    Plot the distribution of homophily ratios across different stages.

    Parameters:
    - graphs (list of networkx.Graph): List of three graphs corresponding to different stages.
    - stages (list of str): List of stage names for labeling the plot.
    - dataset (str): Name of the dataset for saving the plot file.

    This function calculates homophily ratios for each graph and plots their distributions using KDE plots.
    The plot is saved to a file with a name based on the dataset.
    """
    if len(graphs) != 3 or len(stages) != 3:
        raise ValueError("You must provide exactly three graphs and three stages.")
    
    homophily_data = [calculate_homophily_ratios(graph) for graph in graphs]
    colors = ['#2E8B57', '#FF6347', '#8670FD']  # Different colors for better visual distinction

    plt.figure(figsize=(12, 6))

    for i in range(3):
        sns.kdeplot(homophily_data[i], color=colors[i], fill=True, label=f'{stages[i]}', common_norm=True)

    plt.xlabel('Homophily Ratio')
    plt.ylabel('Density')
    plt.title('Homophily Ratios Distribution Across Stages')
    plt.legend(loc='upper left')
    plt.grid(True)

    # Save the plot to a file
    output_file = f'./results/gnn/images/gin_distribution_homophily_{dataset}.png'
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()

In [21]:
def plot_degree_distribution(G_old, G_new, output_file='gin_plot_degree_distributions.png'):
    """
    Plot and compare the degree distributions of two graphs.

    Parameters:
    - G_old (networkx.Graph): The first graph for comparison.
    - G_new (networkx.Graph): The second graph for comparison.
    - output_file (str): Name of the file where the plot will be saved.

    This function plots the degree distributions of the two graphs on the same plot for comparison and saves the plot to a file.
    """
    output_file = './results/gnn/images/' + output_file

    # Get degree sequences for both graphs and sort them
    degree_sequence_old = sorted((d for n, d in G_old.degree()), reverse=True)
    degree_sequence_new = sorted((d for n, d in G_new.degree()), reverse=True)

    plt.figure(figsize=(10, 6))

    # Plot the degree distribution for the old graph
    plt.plot(degree_sequence_old, 'o', label='G_old', alpha=0.5)
    
    # Plot the degree distribution for the new graph
    plt.plot(degree_sequence_new, 'o', label='G_new', alpha=0.5)

    plt.title("Degree Distribution Comparison")
    plt.xlabel("Rank Node")
    plt.ylabel("Degree")
    plt.yscale('log')
    plt.legend()

    # Save the plot to a file
    plt.savefig(output_file)
    plt.close()

## Model

In [22]:
"""
    GIN: Graph Isomorphism Networks
    HOW POWERFUL ARE GRAPH NEURAL NETWORKS? (Keyulu Xu, Weihua Hu, Jure Leskovec and Stefanie Jegelka, ICLR 2019)
    https://arxiv.org/pdf/1810.00826.pdf
"""

class GIN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers, in_channels, 
                 out_channels, batch_norm=False, dropout=0.0, 
                 drop_input=False, residual=False, graph_task=False):
        """
        Initialize the Graph Isomorphism Network (GIN) model.

        Args:
            hidden_channels (int): Number of hidden channels in MLP layers.
            num_layers (int): Number of GIN layers in the network.
            in_channels (int): Number of input features.
            out_channels (int): Number of output features.
            batch_norm (bool, optional): Whether to apply batch normalization. Default is False.
            dropout (float, optional): Dropout rate for dropout layers. Default is 0.0.
            drop_input (bool, optional): Whether to apply dropout to input features. Default is False.
            residual (bool, optional): Whether to use residual connections. Default is False.
            graph_task (bool, optional): Whether the task is a graph-level task. Default is False.
        """
        super().__init__()
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.batch_norm = batch_norm
        self.dropout = dropout
        self.drop_input = drop_input
        self.residual = residual
        self.graph_task = graph_task
        
        self.mlp_layers = torch.nn.ModuleList()  # List to store MLP layers
        self.batch_norm_layers = torch.nn.ModuleList()  # List to store batch normalization layers
        
        # Adding input layer MLP
        if residual:
            self.mlp_layers.append(torch.nn.Sequential(
                torch.nn.Linear(2 * in_channels, hidden_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(hidden_channels, hidden_channels)
            ))
        else:
            self.mlp_layers.append(torch.nn.Sequential(
                torch.nn.Linear(in_channels, hidden_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(hidden_channels, hidden_channels)
            ))
        if self.batch_norm:
            self.batch_norm_layers.append(torch.nn.BatchNorm1d(hidden_channels))
            
        # Adding hidden layer MLPs
        for i in range(num_layers - 2):
            if residual:
                self.mlp_layers.append(torch.nn.Sequential(
                    torch.nn.Linear(2 * hidden_channels, hidden_channels),
                    torch.nn.ReLU(),
                    torch.nn.Linear(hidden_channels, hidden_channels)
                ))
            else:
                self.mlp_layers.append(torch.nn.Sequential(
                    torch.nn.Linear(hidden_channels, hidden_channels),
                    torch.nn.ReLU(),
                    torch.nn.Linear(hidden_channels, hidden_channels)
                ))
            if self.batch_norm:
                self.batch_norm_layers.append(torch.nn.BatchNorm1d(hidden_channels))
        
        # Adding output layer MLP
        if residual:
            self.mlp_layers.append(torch.nn.Sequential(
                torch.nn.Linear(2 * hidden_channels, hidden_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(hidden_channels, out_channels)
            ))
        else:
            self.mlp_layers.append(torch.nn.Sequential(
                torch.nn.Linear(hidden_channels, hidden_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(hidden_channels, out_channels)
            ))

    def forward(self, x, adj):
        """
        Forward pass of the GIN model.

        Args:
            x (Tensor): Input feature matrix.
            adj (Tensor): Adjacency matrix for graph convolution.

        Returns:
            Tensor: Output feature matrix or graph-level output based on graph_task.
        """
        # Apply dropout to input features if drop_input is set to True
        if self.drop_input:
            x = F.dropout(x, p=self.dropout, training=self.training)
            
        for i in range(self.num_layers - 1):  # Exclude output layer
            # Aggregation phase: Compute graph convolution and add residual connections
            if self.residual:
                x = torch.cat((adj @ x + x, x), 1)
            else:
                x = adj @ x + x
            
            # Transformation phase: Apply MLP layer and optional batch normalization
            x = self.mlp_layers[i](x)
            if self.batch_norm:
                x = self.batch_norm_layers[i](x)
            x = F.relu(x)  # Activation function
            x = F.dropout(x, p=self.dropout, training=self.training)  # Dropout
        
        # Aggregation phase for the output layer
        if self.residual:
            x = torch.cat((adj @ x + x, x), 1)
        else:
            x = adj @ x + x
        
        # Transformation phase for the output layer
        x = self.mlp_layers[-1](x)

        # Graph-level readout if graph_task is True
        if self.graph_task:
            x = torch.mean(x, dim=0)  # Aggregate node features to a single graph-level feature
        
        return x

## Train

In [23]:
def full_batch_step(model, optimizer, criterion, x_train, y_train, 
                    adj_train, train_mask, logging = False, adj_cond = None):
    """
    Perform a single optimization step on the model using a full batch of training data.

    Args:
        model (torch.nn.Module): The model to be trained.
        optimizer (torch.optim.Optimizer): The optimizer used for updating model parameters.
        criterion (callable): The loss function.
        x_train (torch.Tensor): The input features for the training data.
        y_train (torch.Tensor): The target labels for the training data.
        adj_train (torch.Tensor): The adjacency matrix for the training data.
        train_mask (torch.Tensor or None): Mask indicating which nodes to consider for training.
        logging (bool, optional): Whether to log training metrics. Defaults to False.
        adj_cond (torch.Tensor or None, optional): The condensed adjacency matrix. Defaults to None.
    Returns:
        loss (torch.Tensor): The computed loss for the current batch.
    """
    model.train()
    optimizer.zero_grad()

    if adj_cond:
        out = model(x_train, adj_train, adj_cond)
    else:
        out = model(x_train, adj_train)
    if train_mask == None:
        loss = criterion(out, y_train)
    else:
        loss = criterion(out[train_mask], y_train[train_mask])
    loss.backward()
    optimizer.step()

    if logging:
        acc,micro_f1,sens,spec = metrics(out,y_train)
        print(f"Train accuracy: {acc}, Train micro_f1: {micro_f1},Train Sens: {sens}, Train Spec: {spec}")

    return loss

In [24]:
def evaluate(model, x, y, adj, mask, adj_cond = None):
    """
    Evaluate the model on the provided data.

    Args:
        model (torch.nn.Module): The model to be evaluated.
        x (torch.Tensor): The input features for the evaluation data.
        y (torch.Tensor): The target labels for the evaluation data.
        adj (torch.Tensor): The adjacency matrix for the evaluation data.
        mask (torch.Tensor): Mask indicating which nodes to consider for evaluation.
        adj_cond (torch.Tensor or None, optional): The condensed adjacency matrix. Defaults to None.

    Returns:
        acc (float): The accuracy of the model on the evaluation data.
        micro_f1 (float): The micro-averaged F1 score on the evaluation data.
        sens (float): The sensitivity (recall) of the model on the evaluation data.
        spec (float): The specificity of the model on the evaluation data.
    """
    with torch.no_grad():
        model.eval()
        if adj_cond:
            out = model(x, adj, adj_cond).squeeze()
        else:
            out = model(x, adj).squeeze()
        acc,micro_f1,sens,spec = metrics(out[mask],y[mask])
    
    return acc, micro_f1, sens, spec


In [25]:
def train(model, device, x_train, y_train, adj_train, adj_train_cond = None, train_mask = None, x_val = None, 
          y_val = None, adj_val = None, adj_val_cond = None, val_mask = None, x_test = None, 
          y_test = None, adj_test = None, adj_test_cond = None, test_mask = None, multilabel = True, 
          lr = 0.0005, num_epoch = 100):
    """
    Train a model using the provided training data and evaluate it on validation and test data.

    Args:
        model (torch.nn.Module): The model to be trained.
        device (torch.device): The device (CPU or GPU) to run the computations on.
        x_train (torch.Tensor): The input features for the training data.
        y_train (torch.Tensor): The target labels for the training data.
        adj_train (torch.Tensor): The adjacency matrix for the training data.
        adj_train_cond (torch.Tensor or None, optional): The condensed adjacency matrix for training. Defaults to None.
        train_mask (torch.Tensor or None, optional): Mask indicating which nodes to consider for training. Defaults to None.
        x_val (torch.Tensor or None, optional): The input features for the validation data. Defaults to None.
        y_val (torch.Tensor or None, optional): The target labels for the validation data. Defaults to None.
        adj_val (torch.Tensor or None, optional): The adjacency matrix for the validation data. Defaults to None.
        adj_val_cond (torch.Tensor or None, optional): The condensed adjacency matrix for validation. Defaults to None.
        val_mask (torch.Tensor or None, optional): Mask indicating which nodes to consider for validation. Defaults to None.
        x_test (torch.Tensor or None, optional): The input features for the test data. Defaults to None.
        y_test (torch.Tensor or None, optional): The target labels for the test data. Defaults to None.
        adj_test (torch.Tensor or None, optional): The adjacency matrix for the test data. Defaults to None.
        adj_test_cond (torch.Tensor or None, optional): The condensed adjacency matrix for testing. Defaults to None.
        test_mask (torch.Tensor or None, optional): Mask indicating which nodes to consider for testing. Defaults to None.
        multilabel (bool, optional): Whether the task is multilabel classification. Defaults to True.
        lr (float, optional): Learning rate for the optimizer. Defaults to 0.0005.
        num_epoch (int, optional): Number of training epochs. Defaults to 100.
    Returns:
        tuple: A tuple containing:
            - max_val_acc (float): Best validation accuracy achieved.
            - max_val_f1 (float): Best validation F1 score achieved.
            - max_val_sens (float): Best validation sensitivity achieved.
            - max_val_spec (float): Best validation specificity achieved.
            - max_val_test_acc (float): Best test accuracy achieved.
            - max_val_test_f1 (float): Best test F1 score achieved.
            - max_val_test_sens (float): Best test sensitivity achieved.
            - max_val_test_spec (float): Best test specificity achieved.
            - session_memory (float): Peak GPU memory usage during the session (in MB).
            - train_memory (float): Peak GPU memory usage during training (in MB).
            - train_time_avg (float): Average time per epoch during training.
    """

    # passing model and training data to GPU
    model = model.to(device)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    adj_train = adj_train.to(device)
    if adj_train_cond:
        adj_train_cond = adj_train_cond.to(device)
    if train_mask != None:
        train_mask = train_mask.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    if multilabel:
        criterion = torch.nn.BCEWithLogitsLoss()
    else:
        criterion = torch.nn.CrossEntropyLoss()
    
    max_val_acc = 0
    max_val_sens = 0
    max_val_spec = 0
    max_val_f1 = 0
    max_val_test_acc = 0
    max_val_test_sens = 0
    max_val_test_spec = 0
    max_val_test_f1 = 0
    
    time_arr = np.zeros((num_epoch,))

    for epoch in range(num_epoch):
            
        # single mini batch step
        t = time.time()

        loss = full_batch_step(model, optimizer, criterion, 
                                   x_train, y_train, adj_train, train_mask, 
                                   logging=False, adj_cond=adj_train_cond)
        
        time_per_epoch = time.time() - t
        time_arr[epoch] = time_per_epoch
        
        if epoch == 0:
            train_memory = torch.cuda.max_memory_allocated(device)*2**(-20)
            
            # passing validation and test data to GPU (we do it after first forward pass to get)
            # accurate pure training GPU memory usage
            if x_val != None and y_val != None and adj_val != None and val_mask != None:
                x_val = x_val.to(device)
                y_val = y_val.to(device)
                adj_val = adj_val.to(device)
                val_mask = val_mask.to(device)
                if adj_val_cond:
                    adj_val_cond = adj_val_cond.to(device)
                if x_test != None and y_test != None and adj_test != None and test_mask != None:
                    x_test = x_test.to(device)
                    y_test = y_test.to(device)
                    adj_test = adj_test.to(device)
                    test_mask = test_mask.to(device)
                    if adj_test_cond:
                        adj_test_cond = adj_test_cond.to(device)
        
        if epoch % 10 == 0:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.10f}, training time: {time_per_epoch:.5f}')
            print(f"Peak GPU Memory Usage: {torch.cuda.max_memory_allocated(device)*2**(-20)} MB")
        
        # evaluation
        if x_val != None and y_val != None:
            acc, micro_f1, sens, spec = evaluate(model, x_val, y_val, adj_val, 
                                                 val_mask, adj_cond = adj_val_cond)
            
            if epoch % 100 == 0:
                print(f"Val accuracy: {acc}, Val micro_f1: {micro_f1}, Val Sens: {sens}, Val Spec: {spec}")
            
            if acc > max_val_acc:
                max_val_acc = acc
                max_val_f1 = micro_f1
                max_val_sens = sens
                max_val_spec = spec
                
                if (x_test != None and y_test != None):
                    acc, micro_f1, sens, spec = evaluate(model, x_test, y_test, 
                                                         adj_test, test_mask, adj_cond = adj_test_cond)
                    max_val_test_acc = acc
                    max_val_test_f1 = micro_f1
                    max_val_test_sens = sens
                    max_val_test_spec = spec
                    
                    print("===========================================Best Model Update:=======================================")
                    print(f"Val accuracy: {max_val_acc}, Val f1: {max_val_f1}, Val Sens: {max_val_sens}, Val Spec: {max_val_spec}")
                    print(f"Test accuracy: {max_val_test_acc}, Test f1: {max_val_test_f1}, Test Sens: {max_val_test_sens}, Test Spec: {max_val_test_spec}")
                    print("====================================================================================================")

    print("Best Model:")
    print(f"Val accuracy: {max_val_acc}, Val f1: {max_val_f1}, Val Sens: {max_val_sens}, Val Spec: {max_val_spec}")
    print(f"Test accuracy: {max_val_test_acc}, Test f1: {max_val_test_f1}, Test Sens: {max_val_test_sens}, Test Spec: {max_val_test_spec}")
    print(f"Average time per epoch: {time_arr[10:].mean()}") # don't include the first few epoch (slower due to Torch initialization)
    print(f"Training GPU Memory Usage: {train_memory} MB")
    print(f"Peak GPU Memory Usage: {torch.cuda.max_memory_allocated(device)*2**(-20)} MB")
    
    # cleaning memory and stats
    session_memory = torch.cuda.max_memory_allocated(device)*2**(-20)
    train_time_avg = time_arr[10:].mean()
    del x_val
    del y_val
    del x_test
    del y_test
    model = model.to('cpu')
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats(device)
    
    return (max_val_acc, max_val_f1, max_val_sens, max_val_spec, max_val_test_acc,
            max_val_test_f1, max_val_test_sens, max_val_test_spec, session_memory, 
            train_memory, train_time_avg)

## Calculate Mean and Variance

In [26]:
def calculate_mean_and_variance(filename):
    """
    Calculates mean and variance of specified columns grouped by unique running settings.

    Args:
        filename (str): Path to the CSV file containing the data.
    """
    # Read the data from a CSV file
    data = pd.read_csv(filename)

    # Define the columns that represent the unique running setting
    grouping_columns = ['Dataset', 'Model_Type', 'Node_Num', 'Hidden_Dimension', 'Max_Communities', 'Remove_Edges', 'Epochs','Topological_Measure','Make_Unbalanced','Dense']

    # Define the columns for which we want to calculate the mean and variance
    measurement_columns = ['Session_Memory', 'Train_Memory', 'Train_Time_Avg', 'Max_Val_Acc', 'Max_Val_F1', 
                           'Max_Val_Sens', 'Max_Val_Spec', 'Max_Val_Test_Acc', 'Max_Val_Test_F1', 
                           'Max_Val_Test_Sens', 'Max_Val_Test_Spec']

    # Group the data by the unique running setting
    grouped_data = data.groupby(grouping_columns)

    # Calculate the mean and variance for each group
    mean_data = grouped_data[measurement_columns].mean().reset_index()
    variance_data = grouped_data[measurement_columns].std().reset_index()

    # Calculate the number of rows in each group
    count_data = grouped_data.size().reset_index(name='Count')

    # Merge the mean, variance, and count data
    mean_data = mean_data.merge(count_data, on=grouping_columns)
    variance_data = variance_data.merge(count_data, on=grouping_columns)

    # Print the results
    print("Mean values for each group with counts:\n", mean_data)
    print("\nVariance values for each group with counts:\n", variance_data)


In [27]:
def execute_model(tp_measure="none"):
    topological_measure = tp_measure

    (x, y, edge_index, train_mask, val_mask, test_mask) = get_dataset(
        sparse=not dense, balanced=not make_unbalanced
    )

    num_features = x.shape[1]
    num_classes = int(max(y) + 1)

    # Construct networkx graph
    G = construct_graph(x, y, edge_index, train_mask, val_mask, test_mask)

    curvature = FormanRicci(G)
    G_topo = G.copy()
    adj_train_cond = None
    adj_val_cond = None
    adj_test_cond = None

    # Split the graph to train, val, and test (inductive training)
    (
        x_train,
        y_train,
        edge_train,
        train_mask,
        x_val,
        y_val,
        edge_val,
        val_mask,
        x_test,
        y_test,
        edge_test,
        test_mask,
    ) = split_graph(G, multilabel=multilabel)

    if topological_measure != "none":
        graphs = []
        stages = []
        output_file = f"./results/gnn/images/degree_distribution_{model_type}_topological_measure.png"
        G_old = G_topo

        if topological_measure == "curvature":
            curvature.compute_ricci_curvature()
            G_topo = curvature.G
            # Rename edge/node attribute 'formanCurvature' to 'topo'
            for node in G_topo.nodes():
                if "formanCurvature" in G_topo.nodes[node]:
                    G_topo.nodes[node]["topo"] = G_topo.nodes[node].pop(
                        "formanCurvature"
                    )
            for u, v in G_topo.edges():
                if "formanCurvature" in G_topo[u][v]:
                    G_topo[u][v]["topo"] = G_topo[u][v].pop("formanCurvature")
        elif topological_measure == "degree_centrality":
            nodes_topo_results = nx.degree_centrality(G_topo)
            edges_topo_results = visual_utils.edge_degree_centrality(G_topo)
            for node in G_topo.nodes():
                G_topo.nodes[node]["topo"] = nodes_topo_results[node]
            for edge, degree_centrality in edges_topo_results.items():
                G_topo[edge[0]][edge[1]]["topo"] = degree_centrality
        elif topological_measure == "betweenness_centrality":
            nodes_topo_results = nx.betweenness_centrality(G_topo)
            edges_topo_results = nx.edge_betweenness_centrality(G_topo)
            for node in G_topo.nodes():
                G_topo.nodes[node]["topo"] = nodes_topo_results[node]
            for u, v in G_topo.edges():
                G_topo[u][v]["topo"] = edges_topo_results[(u, v)]
        elif topological_measure == "eigenvector_centrality":
            nodes_topo_results = nx.eigenvector_centrality(G_topo)
            edges_topo_results = visual_utils.edge_metric_compute(
                nodes_topo_results, G_topo
            )
            for node in G_topo.nodes():
                G_topo.nodes[node]["topo"] = nodes_topo_results[node]
            for edge, degree_centrality in edges_topo_results.items():
                G_topo[edge[0]][edge[1]]["topo"] = degree_centrality
        elif topological_measure == "random":
            for node in G_topo.nodes():
                G_topo.nodes[node]["topo"] = random.random()
            for u, v in G_topo.edges():
                G_topo[u][v]["topo"] = random.random()

        print(f"Number of edges before filtering: {G_topo.number_of_edges()}")

        if remove_edges > 0:
            edge_topo = [(u, v, G_topo[u][v]["topo"]) for u, v in G_topo.edges()]
            edge_topo.sort(key=lambda x: x[2])
            edges_to_remove_high = edge_topo[-remove_edges:]
            edges_to_remove_low = edge_topo[:remove_edges]
            edges_to_remove = edges_to_remove_low
            G_topo.remove_edges_from([(u, v) for u, v, _ in edges_to_remove])

        print(f"Number of edges after filtering: {G_topo.number_of_edges()}")

    # Normalize Adjacency Matrices
    adj_train = construct_normalized_adj(edge_train, x_train.shape[0])
    adj_val = construct_normalized_adj(edge_val, x_val.shape[0])
    adj_test = construct_normalized_adj(edge_test, x_test.shape[0])

    # Convert feature and labels to torch tensor
    x_train = torch.tensor(x_train, dtype=torch.float32)
    y_train = torch.tensor(y_train)
    x_val = torch.tensor(x_val, dtype=torch.float32)
    y_val = torch.tensor(y_val)
    val_mask = torch.tensor(val_mask)
    x_test = torch.tensor(x_test, dtype=torch.float32)
    y_test = torch.tensor(y_test)
    test_mask = torch.tensor(test_mask)

    model = GIN(
        hidden_channels=hidden_channels,
        num_layers=layers,
        in_channels=x_train.shape[1],
        out_channels=num_classes,
        batch_norm=batch_norm,
        dropout=dropout,
        residual=residual,
    )

    print(model)

    if do_evaluation:
        (
            max_val_acc,
            max_val_f1,
            max_val_sens,
            max_val_spec,
            max_val_test_acc,
            max_val_test_f1,
            max_val_test_sens,
            max_val_test_spec,
            session_memory,
            train_memory,
            train_time_avg,
        ) = train(
            model,
            device,
            x_train=x_train,
            y_train=y_train,
            adj_train=adj_train,
            x_val=x_val,
            y_val=y_val,
            adj_val=adj_val,
            val_mask=val_mask,
            x_test=x_test,
            y_test=y_test,
            adj_test=adj_test,
            test_mask=test_mask,
            multilabel=multilabel,
            lr=lr,
            num_epoch=num_epoch,
        )
    else:
        (
            max_val_acc,
            max_val_f1,
            max_val_sens,
            max_val_spec,
            max_val_test_acc,
            max_val_test_f1,
            max_val_test_sens,
            max_val_test_spec,
            session_memory,
            train_memory,
            train_time_avg,
        ) = train(
            model,
            device,
            x_train=x_train,
            y_train=y_train,
            adj_train=adj_train,
            x_val=None,
            y_val=None,
            adj_val=None,
            val_mask=None,
            x_test=None,
            y_test=None,
            adj_test=None,
            test_mask=None,
            multilabel=multilabel,
            lr=lr,
            num_epoch=num_epoch,
        )


In [28]:
execute_model()

Loading Dataset
=============== Dataset Types ==============
Type of features: <class 'numpy.ndarray'>
Type of labels: <class 'numpy.ndarray'>
Type of edge_index: <class 'numpy.ndarray'>
Type of train_mask: <class 'torch.Tensor'>
Type of val_mask: <class 'torch.Tensor'>
Type of test_mask: <class 'torch.Tensor'>
=============== Dataset Properties ==================
Total Nodes: 6515
Total Edges: 2063176
Number of Features: 150528
Number of Labels: 6
Task Type: Multi-class Classification
Training Nodes: 5352
Validation Nodes: 581
Testing Nodes: 582

[Before unbalancing] Class distribution in the training set:
Label 0: 900 samples
Label 1: 792 samples
Label 2: 1000 samples
Label 3: 1000 samples
Label 4: 660 samples
Label 5: 1000 samples
[Before unbalancing] Class distribution in the validation set:
Label 0: 100 samples
Label 1: 99 samples
Label 2: 100 samples
Label 3: 100 samples
Label 4: 82 samples
Label 5: 100 samples
[Before unbalancing] Class distribution in the test set:
Label 0: 100

KeyboardInterrupt: 

In [ ]:
#execute_model(tp_measure="curvature")

In [ ]:
#execute_model(tp_measure="degree_centrality")

In [ ]:
#execute_model(tp_measure="betweenness_centrality")

In [ ]:
#execute_model(tp_measure="eigenvector_centrality")